### TODO

kfold ++ 

Image augmentation (https://keras.io/preprocessing/image/, https://www.kaggle.com/stefanie04736/simple-keras-model-with-k-fold-cross-validation)

weight normalization

batch-normalization

resnet - rcnn

In [114]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [115]:
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
VAL_SIZE = 0.2
RANDOM_STATE = 2018 #seed
#Model
NO_EPOCHS = 1
BATCH_SIZE = 128

IS_LOCAL = False

import os
PATH="./fashionmnist/"
print(os.listdir(PATH))

['t10k-images-idx3-ubyte', 'fashion-mnist_test.csv', 't10k-labels-idx1-ubyte', 'train-images-idx3-ubyte', 'fashion-mnist_train.csv', 'train-labels-idx1-ubyte']


In [116]:
train_file = PATH+"fashion-mnist_train.csv"
test_file  = PATH+"fashion-mnist_test.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

In [117]:
print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1])

Fashion MNIST train -  rows: 60000  columns: 785
Fashion MNIST test -  rows: 10000  columns: 785


In [118]:
# data preprocessing
def data_preprocessing(raw):
    out_y = keras.utils.to_categorical(raw.label, NUM_CLASSES)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

In [119]:
# prepare the data
X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)


In [120]:
print('X train len:', X.shape[1:4])
X[0:5].shape[0]
len(X[0:5])

X train len: (28, 28, 1)


5

In [68]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_STATE)

In [121]:
# Model
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(model, X, y, cv=5, scoring='accuracy') 
# "cross_val_score" method calls model(estimator).fit --> https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_validation.py


TypeError: Cannot clone object '<tensorflow.python.keras.engine.sequential.Sequential object at 0x13f5bd320>' (type <class 'tensorflow.python.keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

### Kfold CV added.

In [122]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=VAL_SIZE, random_state=RANDOM_STATE)
sss.get_n_splits(X, y)
print(sss)
fold_no=1
for train_index, test_index in sss.split(X, y):
    print("\nFold_no #{}".format(fold_no))
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    print("Fashion MNIST train -  rows:",X_train.shape[0]," columns:", X_train.shape[1:4])
    print("Fashion MNIST valid -  rows:",X_val.shape[0]," columns:", X_val.shape[1:4])
    print("Fashion MNIST test -  rows:",X_test.shape[0]," columns:", X_test.shape[1:4])
    print('X val len:', X_val.shape)
    train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))
    
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    
    last_epoch_no = NO_EPOCHS - 1
    acc_fin = acc[last_epoch_no]
    val_acc_fin = val_acc[last_epoch_no]
    loss_fin = loss[last_epoch_no]
    val_loss_fin = val_loss[last_epoch_no]
    
    print("Train acc: {}\t Validation acc: {}\t Train loss: {}\t Validation loss: {}".format(acc_fin,val_acc_fin,loss_fin,val_loss_fin))
    
    

StratifiedShuffleSplit(n_splits=5, random_state=2018, test_size=0.2,
            train_size=None)

Fold_no #1
TRAIN: [48375 50843 43640 ...   465 35707   130] TEST: [ 2860 21189 13884 ... 27507 52393 27083]
Fashion MNIST train -  rows: 48000  columns: (28, 28, 1)
Fashion MNIST valid -  rows: 12000  columns: (28, 28, 1)
Fashion MNIST test -  rows: 10000  columns: (28, 28, 1)
X val len: (12000, 28, 28, 1)
Train on 48000 samples, validate on 12000 samples
22144/48000 [============>.................] - ETA: 18s - loss: 0.6595 - acc: 0.7594

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
   
    fig = tools.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])

    
    iplot(fig, filename='accuracy-loss')

plot_accuracy_and_loss(train_model)